In [0]:
import datetime
from pyspark.sql.functions import col,concat_ws
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

print(datetime.datetime.now())

2025-02-12 16:47:57.548127


In [0]:
# Mount storage
# dbutils.fs.mount(
#     source='wasbs://ipaakashformula1project@dp203storageaccount22.blob.core.windows.net',  # protocol and account name are required
#     mount_point='/mnt/ipaakashformula1project',  # mount point basically path
#     extra_configs={'fs.azure.account.key.dp203storageaccount22.blob.core.windows.net': "0pLjWY6gohOdXM9D2/yvvdK2jR5kQAAve+MKmKqKsrn+zQGXNUU7tsUNrU+J/aPVSYKiS3Rdyuvg+AStKb60fg=="}  # access key of storage account
# )

In [0]:
# circuits df
circuitsdf = spark.read.csv('/mnt/ipaakashformula1project/ingestdata/Datasets/circuits.csv', header=True,inferSchema=True)
circuitsdf = circuitsdf.withColumnRenamed('circuitId', 'curcuit_id')
circuitsdf = circuitsdf.withColumnRenamed('circuitRef', 'circuit_ref')
circuitsdf = circuitsdf.withColumnRenamed('lat', 'latitude')
circuitsdf = circuitsdf.withColumnRenamed('long', 'longitude')
circuitsdf = circuitsdf.withColumnRenamed('alt', 'altitude')
circuitsdf =circuitsdf.drop('Url')
circuitsdf = circuitsdf.withColumn('ingestion_data', current_timestamp())
circuitsdf.coalesce(1).write.mode('overwrite').format('delta').save('/mnt/ipaakashformula1project/clean_data/circuits')



In [0]:
# race df
races_df = spark.read.csv('/mnt/ipaakashformula1project/ingestdata/Datasets/races.csv', header=True,inferSchema=True)
races_df = races_df.withColumnRenamed('raceId', 'race_id')
races_df = races_df.withColumnRenamed('year', 'race_year')
races_df = races_df.withColumnRenamed('circuitId', 'circuit_id')

races_df =races_df.drop('Url')
races_df =races_df.drop('time')
races_df =races_df.drop('date')
races_df = races_df.withColumn('race_timestamp', current_timestamp())
races_df.write.mode('overwrite').partitionBy('race_year').format('delta').save('/mnt/ipaakashformula1project/clean_data/races')



In [0]:
# constructors.json
constructors_df = spark.read.option("inferSchema", "true").json('/mnt/ipaakashformula1project/ingestdata/Datasets/constructors.json')
# constructors_df.show()
constructors_df = constructors_df.withColumnRenamed('constructorId', 'constructor_id')
constructors_df = constructors_df.withColumnRenamed('constructorRef', 'constructor_ref')
constructors_df =constructors_df.drop('Url')
constructors_df = constructors_df.withColumn('ingestion_data', current_timestamp())
constructors_df.coalesce(1).write.mode('overwrite').format('delta').save('/mnt/ipaakashformula1project/clean_data/constructors')



In [0]:
# drivers.json
drivers_df = spark.read.option("inferSchema", "true").json('/mnt/ipaakashformula1project/ingestdata/Datasets/drivers.json')
# drivers_df.show()
drivers_df = drivers_df.withColumnRenamed('driverId', 'driver_id')
drivers_df = drivers_df.withColumnRenamed('driverRef', 'driver_ref')
drivers_df = drivers_df.withColumn("name", concat_ws(" ", col("name.forename"), col("name.surname")))
# drivers_df.show()

drivers_df =drivers_df.drop('Url')
drivers_df =drivers_df.drop('name.forename')
drivers_df =drivers_df.drop('name.surname')

drivers_df = constructors_df.withColumn('ingestion_data', current_timestamp())

drivers_df.coalesce(1).write.mode('overwrite').format('delta').save('/mnt/ipaakashformula1project/clean_data/drivers')



In [0]:
# result.json
results_df = spark.read.option("inferSchema", "true").json('/mnt/ipaakashformula1project/ingestdata/Datasets/results.json')
# drivers_df.show()
results_df = results_df.withColumnRenamed('resultId', 'result_id')
results_df = results_df.withColumnRenamed('raceId', 'race_id')
results_df = results_df.withColumnRenamed('driverId', 'driver_id')
results_df = results_df.withColumnRenamed('ConstructorId', 'constructor_id')
results_df = results_df.withColumnRenamed('positionText', 'position_text')
results_df = results_df.withColumnRenamed('positionOrder', 'position_order')
results_df = results_df.withColumnRenamed('fastestLap', 'fastest_lap')
results_df = results_df.withColumnRenamed('fastestLapTime', 'fastest_lap_time')
results_df = results_df.withColumnRenamed('FastestLapSpeed', 'fastest_lap_speed')


# drivers_df.show()

results_df =results_df.drop('“statusId”')


results_df = results_df.withColumn('ingestion_data', current_timestamp())

results_df.write.mode('overwrite').partitionBy('race_id').format('delta').save('/mnt/ipaakashformula1project/clean_data/results')



In [0]:
# pit_stops.json

# Define schema for pit_stops.json
pit_stops_schema = StructType([
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("stop", IntegerType(), True),
    StructField("lap", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

# Read JSON file with schema
pit_stops_df = spark.read.schema(pit_stops_schema).json('/mnt/ipaakashformula1project/ingestdata/Datasets/pit_stops.json')

pit_stops_df = pit_stops_df.withColumnRenamed('driverId', 'driver_id')
pit_stops_df = pit_stops_df.withColumnRenamed('raceId', 'race_id')

pit_stops_df = pit_stops_df.withColumn('ingestion_data', current_timestamp())

pit_stops_df.write.mode('overwrite').format('delta').save('/mnt/ipaakashformula1project/clean_data/pit_stops')



In [0]:
# laps time df
laps_times_df = spark.read.csv('/mnt/ipaakashformula1project/ingestdata/Datasets/lap_times/*.csv', header=True,inferSchema=True)
laps_times_df = laps_times_df.withColumnRenamed('raceId', 'race_id')
laps_times_df = laps_times_df.withColumnRenamed('driverId', 'driver_id')
laps_times_df = laps_times_df.withColumn('ingestion_data', current_timestamp())
laps_times_df.coalesce(1).write.mode('overwrite').format('delta').save('/mnt/ipaakashformula1project/clean_data/laps_times')



In [0]:
# laps time df
qualifying_schema = StructType([
    StructField("qualifyId", IntegerType(), True),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("q1", DoubleType(), True),
        StructField("q2", DoubleType(), True),
            StructField("q3", DoubleType(), True)
])
qualifying_df = spark.read.schema(qualifying_schema).json('/mnt/ipaakashformula1project/ingestdata/Datasets/qualifying/*.json')


qualifying_df = qualifying_df.withColumnRenamed('qualifyingId', 'qualifying_id')
qualifying_df = qualifying_df.withColumnRenamed('raceId', 'race_id')
qualifying_df = qualifying_df.withColumnRenamed('constructorId', 'constructor_id')
qualifying_df = qualifying_df.withColumnRenamed('driverId', 'driver_id')
qualifying_df = qualifying_df.withColumn('ingestion_data', current_timestamp())
qualifying_df.coalesce(1).write.mode('overwrite').format('delta').save('/mnt/ipaakashformula1project/clean_data/qualifying')



In [0]:
print(datetime.datetime.now())

2025-02-12 16:55:37.592301
